In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

In [ ]:
flow_grind_df = pd.read_csv(f'../saved_data/bbr_50ms_2flows/fg_4.csv')

fig = go.Figure()
for flow in flow_grind_df['flow'].unique()[:20]:
    this_flow = flow_grind_df[flow_grind_df['flow'] == flow]
    fig.add_trace(go.Scatter(x = this_flow['time'], 
                             y = this_flow['thru(Mbps)'].ewm(span = 100).mean(), 
                             name = str(flow)))
fig.update_xaxes(title = 'Time (s)')
fig.update_yaxes(title = 'Throughput (Mbps)')
fig.show()

In [ ]:
folder = '../saved_data/bbr_sync_50ms_2flows'

fig = make_subplots(rows=5, cols=2, 
                    x_title = 'Time (s)', y_title = 'Throughput (Mbps)', 
                    shared_xaxes = True)

for run in range(1, 11):
    flow_grind_df = pd.read_csv(f'{folder}/fg_{run}.csv')
    for flow in flow_grind_df['flow'].unique()[:10]:
        this_flow = flow_grind_df[flow_grind_df['flow'] == flow]
        fig.add_trace(go.Scatter(x = this_flow['time'], 
                                 y = this_flow['thru(Mbps)'].ewm(span = 100).mean(), 
                                 name = str(flow)), 
                      row = (run-1)%5 + 1, col = (run-1) // 5 + 1)
# fig.update_xaxes(title = 'Time (s)')
# fig.update_yaxes(title = 'Throughput (Mbps)')
fig.update_yaxes(range = (0, 100))
fig.update(layout_showlegend = False)
fig.show()

In [ ]:
def jains(arr):
    arr = np.array(arr)
    if not arr.any(): ## If all are zeros
        return 1.0
    return arr.mean()**2 / (arr**2).mean()

fig = make_subplots(rows=5, cols=2, 
                    x_title = 'Time (s)', y_title = 'Jains Fairness', 
                    shared_xaxes = True)

for run in range(1, 11):
    flow_grind_df = pd.read_csv(f'{folder}/fg_{run}.csv')
    jains_index = flow_grind_df.groupby('time')['thru(Mbps)'].agg(jains)
    fig.add_trace(go.Scatter(x = jains_index.index, 
                             y = pd.Series(jains_index.values).ewm(span = 100).mean(), 
                             name = str(flow)), 
                  row = (run-1)%5 + 1, col = (run-1) // 5 + 1)
# fig.update_xaxes(title = 'Time (s)')
# fig.update_yaxes(title = 'Throughput (Mbps)')
# fig.update_yaxes(range = (0, 30))
fig.update(layout_showlegend = False)
fig.show()

In [ ]:
## Compare average JFIs
folder1 = '../saved_data/bbr_sync_20ms_10flows'
folder2 = '../saved_data/bbr_20ms_10flows'
cutoff_start_time = 5

def get_mean_JFI(folder):
    jains_indices = []
    for run in range(1, 11):
        flow_grind_df = pd.read_csv(f'{folder}/fg_{run}.csv')
        jains_index = flow_grind_df.groupby('time')['thru(Mbps)'].agg(jains)
        jains_index.index = np.around(jains_index.index, 1)
        jains_indices.append(jains_index)

    mean_j = jains_indices[0]
    ctr = 1
    for j in range(1, len(jains_indices)):
        ctr += 1
        mean_j = mean_j + jains_indices[j]
    mean_j /= ctr
    return mean_j

jfi_1 = get_mean_JFI(folder1)
jfi_2 = get_mean_JFI(folder2)

jfi_1 = jfi_1[jfi_1.index > cutoff_start_time]
jfi_2 = jfi_2[jfi_2.index > cutoff_start_time]

fig = go.Figure()
fig.add_trace(go.Scatter(x = jfi_1.index, 
                         y = pd.Series(jfi_1.values).ewm(span = 100).mean(), 
                         name = folder1))
fig.add_trace(go.Scatter(x = jfi_2.index, 
                         y = pd.Series(jfi_2.values).ewm(span = 100).mean(), 
                         name = folder2))
fig.update_xaxes(title = 'Time (s)')
fig.update_yaxes(title = 'JFI')
fig.show()